In [64]:
from dotenv import load_dotenv
from langchain.chat_models import init_chat_model
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.tools import tool
from langchain.agents import create_agent
from mem0 import MemoryClient
from langchain_openai import OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore
from pinecone import Pinecone
from langchain_community.document_loaders import TextLoader

from langchain_core.documents import Document

import os

load_dotenv()

name = os.getenv("LANGSMITH_PROJECT")
name

'hackathon-rag'

In [65]:
# Load your file
import pandas as pd

df = pd.read_excel("new.xlsx")

def row_to_nl(row):
    """
    Convert one roster row into a meaningful natural-language description
    for RAG embeddings.
    """
    text = (
        f"On {row['Date']} ({row['Day']}), {row['Employee Name']} "
        f"(ID: {row['Employee ID']}) is scheduled for a {row['Hours']}-hour "
        f"{row['Shift Code']} shift from {row['Shift Time']}. "
        f"Employment type: {row['Employment Type']}. "
        f"Status: {row['Status']}. "
        f"Located at Station {row['Station']}, working in Store '{row['Store']}'. "
        f"and the Manager is : {row['Manager']}. "
    )
    
    # Structured metadata for better retrieval
    metadata = {
        "date": row['Date'],
        "employee_name": row['Employee Name'],
        "employee_id": row['Employee ID'],
        "manager": row['Manager'],
        "store": row['Store'],
        "station": row['Station'],
        "shift_code": row['Shift Code']
    }

    return text, metadata

# Apply conversion to every row
nl_chunks = df.apply(row_to_nl, axis=1).tolist()
import json
output_path = "roster_doc.txt"
with open(output_path, "w", encoding="utf-8") as f:
    for text, metadata in nl_chunks:
        combined = (
            f"{text}\n"
            f"Manager: {metadata['manager']} | "
            f"Store: {metadata['store']} | "
            f"Station: {metadata['station']} | "
            f"Employee: {metadata['employee_name']} | "
            f"Date: {metadata['date']}\n"
        )
        f.write(combined + "\n")

nl_chunks[:5]  # show first 5

[("On 2024-12-16 (Monday), John Smith (ID: 1001) is scheduled for a 8.5-hour S shift from 06:30-15:00. Employment type: full-time. Status: Scheduled. Located at Station Kitchen, working in Store 'Store_1'. and the Manager is : Mary Wilson. ",
  {'date': '2024-12-16',
   'employee_name': 'John Smith',
   'employee_id': 1001,
   'manager': 'Mary Wilson',
   'store': 'Store_1',
   'station': 'Kitchen',
   'shift_code': 'S'}),
 ("On 2024-12-16 (Monday), Mike Brown (ID: 1003) is scheduled for a 8.5-hour S shift from 06:30-15:00. Employment type: full-time. Status: Scheduled. Located at Station Kitchen, working in Store 'Store_1'. and the Manager is : Mary Wilson. ",
  {'date': '2024-12-16',
   'employee_name': 'Mike Brown',
   'employee_id': 1003,
   'manager': 'Mary Wilson',
   'store': 'Store_1',
   'station': 'Kitchen',
   'shift_code': 'S'}),
 ("On 2024-12-16 (Monday), Alex Wilson (ID: 1005) is scheduled for a 8.5-hour S shift from 06:30-15:00. Employment type: full-time. Status: Schedu

In [66]:
model = init_chat_model("gpt-4.1")
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

pc = Pinecone()
index = pc.Index("hackathon-manager")

vector_store = PineconeVectorStore(embedding=embeddings, index=index)
vector_store


In [67]:
loader = TextLoader("roster_doc.txt", encoding="utf-8")
docs = loader.load()

print(docs[0].page_content)

On 2024-12-16 (Monday), John Smith (ID: 1001) is scheduled for a 8.5-hour S shift from 06:30-15:00. Employment type: full-time. Status: Scheduled. Located at Station Kitchen, working in Store 'Store_1'. and the Manager is : Mary Wilson. 
Manager: Mary Wilson | Store: Store_1 | Station: Kitchen | Employee: John Smith | Date: 2024-12-16

On 2024-12-16 (Monday), Mike Brown (ID: 1003) is scheduled for a 8.5-hour S shift from 06:30-15:00. Employment type: full-time. Status: Scheduled. Located at Station Kitchen, working in Store 'Store_1'. and the Manager is : Mary Wilson. 
Manager: Mary Wilson | Store: Store_1 | Station: Kitchen | Employee: Mike Brown | Date: 2024-12-16

On 2024-12-16 (Monday), Alex Wilson (ID: 1005) is scheduled for a 8.5-hour S shift from 06:30-15:00. Employment type: full-time. Status: Scheduled. Located at Station Kitchen, working in Store 'Store_1'. and the Manager is : David Miller. 
Manager: David Miller | Store: Store_1 | Station: Kitchen | Employee: Alex Wilson | 

In [68]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
all_splits = text_splitter.split_documents(docs)
print(f"Split blog post into {len(all_splits)} sub-documents.")

Split blog post into 46 sub-documents.


In [69]:
document_ids = vector_store.add_documents(documents=all_splits)

print(document_ids[:3])

['889f0cda-9763-4475-93d5-a1b28bde0d04', 'd719e7a2-0ab4-4c32-83a1-1b9cdc9f03ea', 'ae1e247c-9b19-43e7-9079-918347efc1eb']


In [70]:
@tool(response_format="content_and_artifact")
def retrieve_context(query: str):
    """Retrieve information to help answer a query."""
    # Increase k for queries about managers, stores, or general information
    # to get more comprehensive results
    # Increase k for queries about employees, managers, stores, or general information
    # to get more comprehensive results
    k = 15 if any(keyword in query.lower() for keyword in ['manager', 'store', 'who', 'list', 'all', 'employee', 's manager', 'manager?']) else 10
    retrieved_docs = vector_store.similarity_search(query, k=k)
    serialized = "\n\n".join(
        (f"Source: {doc.metadata}\nContent: {doc.page_content}")
        for doc in retrieved_docs
    )
    return serialized, retrieved_docs



In [71]:
tools = [retrieve_context]
# If desired, specify custom instructions
prompt = (
    """
    You are an intelligent Roster Management Assistant for a 14-day employee schedule system. 
Your job is to accurately answer admin queries using the roster data retrieved from the RAG tool.
Shift Code Metadata:

1F: Front Counter – Morning  
2F: Front Counter – Afternoon  
3F: Front Counter – Evening  
1B: Back Area / Grill – Morning  
2B: Back Area / Grill – Afternoon  
3B: Back Area / Grill – Evening

==========================
 HOW YOU USE THE RAG TOOL
==========================
• You ONLY answer using the context retrieved through the roster-retrieval tool.  
• The retrieval tool returns natural-language descriptions of roster entries for 14 consecutive days.
• If the retrieved context does NOT contain the employee or role the admin is asking for, reply exactly:
  "This role is not filled in the system as of now."

==========================
 HOW YOU UNDERSTAND DATES
==========================
• The roster contains **exact dates** for 14 days (lowest date → highest date).  
• “First Monday”, “Second Friday”, “next Tuesday”, etc. refer to the occurrences **within the 14-day window**, not the calendar month.  
• To interpret “first Monday”, identify the earliest date labeled Monday.  
• To interpret “second Monday”, identify the second occurrence of Monday.  
• When the admin asks about a “week”, treat each consecutive set of 7 days starting from the earliest date as Week 1 and Week 2.

==========================
 HOW TO HANDLE EMPLOYEE QUERIES
==========================
• If an employee name exists in the retrieved data, summarize all relevant shifts:
    - Date
    - Day
    - Shift code / shift time
    - Hours
    - Status
• If the admin asks about:
    - “Who is working on <date/day>?”
    - “Is <employee> free on <date>?”
    - “Who covers the morning shift on the second Wednesday?”
  …use the retrieved roster entries to answer directly and concisely.

==========================
 ANSWERING STYLE
==========================
• Be accurate, structured, and concise and answer in natural language.
• If multiple relevant entries exist, summarize them clearly.
• NEVER make up data. NEVER guess.
• ONLY rely on the retrieved tool output.

==========================
WHEN YOU MUST SAY
"This role is not filled in the system as of now."
==========================
• Name not found in context
• Role not found in context
• No matching shift in context
• No retrieved data at all

==========================
GOAL
==========================
Provide correct, reliable answers about employee schedules, shift details, availability, and roster insights over the 14-day period using ONLY the provided RAG context.

    """
)
agent = create_agent(model, tools, system_prompt=prompt)

In [74]:
query = (
    "who is working on monday afternooshift in kitchen ??"
)

for event in agent.stream(
    {"messages": [{"role": "user", "content": query}]},
    stream_mode="values",
):
    event["messages"][-1].pretty_print()

================================ Human Message =================================

who is working on monday afternooshift in kitchen ??
================================== Ai Message ==================================
Tool Calls:
  retrieve_context (call_ieieciId3jj4GEAHdXP9jyfa)
 Call ID: call_ieieciId3jj4GEAHdXP9jyfa
  Args:
    query: Monday afternoon shift in kitchen
================================= Tool Message =================================
Name: retrieve_context

Source: {'Date': '2024-12-16', 'Day': 'Monday', 'Employee ID': 1003.0, 'Employee Name': 'Mike Brown', 'Employment Type': 'full-time', 'Hours': 8.5, 'Manager': 'Mary Wilson', 'Shift Code': 'S', 'Shift Time': '06:30-15:00', 'Station': 'Kitchen', 'Status': 'Scheduled', 'Store': 'Store_1', 'row_id': 1.0, 'source': 'excel_data', 'start_index': 0.0}
Content: Date: 2024-12-16 | Day: Monday | Store: Store_1 | Employee ID: 1003 | Employee Name: Mike Brown | Station: Kitchen | Shift Code: S | Shift Time: 06:30-15:00 | Hours: 8.